"""
Created on Sun May 28 19:51:21 2023

@author: gakro
"""

this program is to predict the final position of each driver in the race.
thus, the output should be the position (ordinal data from 1 to 20).
the inputs (which parameters to use and which year, race, and session to use)
and the appropriate algorithms should be considered.

training set: Bahrain, Saudi Arabia, Australia, Azerbaijan (all in 2023)
test set: Miami (2023)

In [79]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import fastf1

the following block of code loads the session data of each qualifying race

In [87]:
quali = fastf1.get_session(2022, 1, 4)
quali.load(telemetry=False, laps=False, weather=False)

race = fastf1.get_session(2022, 1, 5)
race.load(telemetry=False, laps=False, weather=False)

core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.0.3]
req            INFO 	Using cached data for driver_info
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '11', '63', '55', '44', '20', '77', '24', '18', '4', '27', '10', '47', '3', '14', '23', '31', '6', '22']
core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.0.3]
req            INFO 	Using cached data for driver_info
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '1', '55', '11', '

In [96]:
print(race._results["Position"])

16   NaN
1    NaN
55   NaN
11   NaN
44   NaN
77   NaN
20   NaN
14   NaN
63   NaN
10   NaN
31   NaN
47   NaN
4    NaN
23   NaN
24   NaN
22   NaN
27   NaN
3    NaN
18   NaN
6    NaN
Name: Position, dtype: float64


In [65]:
bahrain_quali = fastf1.get_session(2023, 1, 4)
bahrain_quali.load(telemetry=False, laps=False, weather=False)

saudi_quali = fastf1.get_session(2023, 2, 4)
saudi_quali.load(telemetry=False, laps=False, weather=False)

australia_quali = fastf1.get_session(2023, 3, 4)
australia_quali.load(telemetry=False, laps=False, weather=False)

azerbaijan_quali = fastf1.get_session(2023, 4, 4)
azerbaijan_quali.load(telemetry=False, laps=False, weather=False)

miami_quali = fastf1.get_session(2023, 5, 4)
miami_quali.load(telemetry=False, laps=False, weather=False)

bahrain_quali_result = bahrain_quali.results.Position
saudi_quali_result = saudi_quali.results.ClassifiedPosition
australia_quali_result = australia_quali.results.ClassifiedPosition
azerbaijan_quali_result = azerbaijan_quali.results.ClassifiedPosition
miami_quali_result = miami_quali.results.ClassifiedPosition

core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.0.3]
req            INFO 	Using cached data for driver_info
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['14', '1', '11', '44', '16', '63', '18', '55', '81', '10', '31', '24', '4', '20', '27', '22', '77', '2', '23', '21']
core           INFO 	Loading data for Saudi Arabian Grand Prix - Qualifying [v3.0.3]
req            INFO 	Using cached data for driver_info
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', 

In [66]:
print(bahrain_quali_result)

14   NaN
1    NaN
11   NaN
44   NaN
16   NaN
63   NaN
18   NaN
55   NaN
81   NaN
10   NaN
31   NaN
24   NaN
4    NaN
20   NaN
27   NaN
22   NaN
77   NaN
2    NaN
23   NaN
21   NaN
Name: Position, dtype: float64


the following block of code loads the session data of each race

In [ ]:
bahrain_race = fastf1.get_session(2023, 1, 5)
bahrain_race.load(telemetry=False, laps=False, weather=False)

saudi_race = fastf1.get_session(2023, 2, 5)
saudi_race.load(telemetry=False, laps=False, weather=False)

australia_race = fastf1.get_session(2023, 3, 5)
australia_race.load(telemetry=False, laps=False, weather=False)

azerbaijan_race = fastf1.get_session(2023, 4, 5)
azerbaijan_race.load(telemetry=False, laps=False, weather=False)

miami_race = fastf1.get_session(2023, 5, 5)
miami_race.load(telemetry=False, laps=False, weather=False)

bahrain_race_result = bahrain_race.results
saudi_race_result = saudi_race.results
australia_race_result = australia_race.results
azerbaijan_race_result = azerbaijan_race.results
miami_race_result = miami_race.results

plot the quali position in the x axis and the race finishing position in the y axis
to see how much they are correlated as the starter.
Try to use the logistic regressio. However, it must be for multiple class
(since what we are predicting is the position in the end of the race from 1 to 20),
and for ordinal output (since the position has the order).

xxx_quali_result.Position => NaN
xxx_quali_result.GridPosition => NaN
xxx_quali_result.ClassifiedPosition => blank
xxx_quali_result.Status => blank

xxx_race_result.Position => blank
xxx_race_result.GridPosition => blank
xxx_race_result.ClassifiedPosition => blank
xxx_race_result.Status => blank

By default, the session results are indexed by driver number and sorted by finishing position.